<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week9/Negation_Rule_based_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import spacy
from collections import defaultdict
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/Model1-Semantic/Jacc_sim_ignore_syllabus.csv")
df = df.drop("Unnamed: 0", axis = 1)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2
0,700295,700313,how many π bonds are present in ferrocene,how many π are present in ferrocene,2073346,2073271
1,700295,731162,how many π bonds are present in ferrocene,how many π bonds are present in silicon dioxide,2073346,1909343
2,698013,697895,what is covalency of chlorine atom in second e...,what is covalency of chlorine atom in second e...,2073324,2073165
3,697895,697895,calculate value of x y here x and y are the to...,calculate value of x y here x and y are the to...,2073215,2073343
4,697895,698013,in phosphorus acid if x is number of non bondi...,in phosphorus acid if x is number of non bondi...,2073202,2073272
...,...,...,...,...,...,...
6639,511206,511067,tim said that he,tim said that he,1551779,1451802
6640,511206,4770,tim said that he,he said that,1551779,494476
6641,511206,4770,tim said that he,he said that,1551779,494307
6642,511206,511067,you look so shabby you,you look so shabby you,1551630,1451801


In [6]:
def calc_curr_to_next(words_ls):
  res = defaultdict(str)
  for index, word in enumerate(words_ls):
    res[word] = words_ls[index+1] if index+1 < len(words_ls) else "*"
  return res

# def calc_curr_to_prev(words_ls):
#   res = defaultdict(str)
#   for index, word in enumerate(words_ls):
#     res[word] = words_ls[index-1] if index > 0 else "*"
#   return res

In [54]:
def generate_list_of_neg_affected_words(ls, contractions, window_size):
  neg_1 = []

  for index in range(len(ls)):
    word = ls[index]
    if word in contractions:
      for j in range(window_size):
        neg_1.append(True)
      index+= (window_size-1)
    else:
      neg_1.append(False)
    
  return neg_1

In [55]:
WINDOW_SIZE = 3

def check_pair_negation(ques1, ques2):
  contractions = ["not", "doesn't", "couldn't", "can't", "wouldn't", "shouldn't", "haven't", "hasn't", "hadn't"]
  wrong_spellings = {"cannot":"can't", "doesnot":"doesn't", "couldnot":"couldn't", "canot":"can't", "wouldnot":"wouldn't", "shouldnot":"shouldn't", "havenot":"haven't", "hasnot":"hasn't", "hadnot":"hadn't"}
  articles = set(['a', 'the', 'an'])


  w1 = ques1.lower().split()
  w2 = ques2.lower().split()

  w1 = [w for w in w1 if w not in articles]
  w2 = [w for w in w2 if w not in articles]

  w1 = [wrong_spellings[w] if w in wrong_spellings else w for w in w1]
  w2 = [wrong_spellings[w] if w in wrong_spellings else w for w in w2]

  words_1 = set(w1)
  words_2 = set(w2)

  # for w in wrong_spellings.keys():
  #   if w in words_1:
  #     words_1.add(wrong_spellings[w])
  #   if w in words_2:
  #    words_2.add(wrong_spellings[w])

  C2N_1 = calc_curr_to_next(w1) 
  C2N_2 = calc_curr_to_next(w2) 

  # for not_w in contractions:
  #   if not_w in words_1 and not_w not in words_2 or not_w in words_2 and not_w not in words_1:
  #     return True
  #   elif not_w in words_1 and C2N_1[not_w] != C2N_2[not_w]: ## skeptical
  #     return True
  #   elif not_w in words_2 and C2N_2[not_w] != C2N_1[not_w]:
  #     return True

  neg_1 = generate_list_of_neg_affected_words(w1, contractions, WINDOW_SIZE)
  neg_2 = generate_list_of_neg_affected_words(w2, contractions, WINDOW_SIZE)

  was_negated_1 = defaultdict(int)
  was_negated_2 = defaultdict(int)

  for index, w in enumerate(w1):
    if neg_1[index]:
      was_negated_1[w] = was_negated_1[w]+ 1

  for index, w in enumerate(w2):
    if neg_2[index]:
      was_negated_2[w] = was_negated_2[w]+ 1

  keys_both = list(was_negated_1.keys())
  keys_both.extend(list(was_negated_2.keys()))

  for key in keys_both:
    if was_negated_1[key] != was_negated_2[key]:
      True


  return False

In [53]:
check_pair_negation("what pairs of compounds are not positional isomers", "what pairs of compounds are not isomers")

False

In [8]:
to_be_appended = []

for index, row in df.iterrows():
  if check_pair_negation(row['ques1'], row['ques2']):
    to_be_appended.append(row)

In [34]:
newDf = pd.DataFrame(to_be_appended)
newDf.to_csv("drive/My Drive/Extramarks Project/Dataset versions/Negation Recognition/Rule_based_v2.csv")
newDf

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2
21,698013,698013,gas that can be collected over water is,gas that cannot be collected over water is,2072295,2072818
22,698013,449048,gas that can be collected over water is,a gas that cannot be collected over water is,2072295,1396453
75,698013,731155,what halides does not hydrolyse at room temper...,what halides does the hydrolyse at room temper...,2071816,1973603
123,700295,3745,paramagnetism is not exhibited by,paramagnetism is exhibited by,2072319,529732
191,698013,698013,what compound does not produce oxyacid of cent...,what compound does not give oxyacid of atom on...,2072261,2071908
...,...,...,...,...,...,...
5504,447545,513355,what is true,what is not true,1354127,1553221
5513,565888,418667,what is not an emulsion,what is an emulsion,1333143,1307991
5514,565888,418750,what is not an emulsion,what is an emulsion,1333143,1303704
5515,565888,4183,what is not an emulsion,what is an emulsion,1333143,543426
